# End-to-end NLP: News Headline classifier

### Setup execution role and session

In [1]:
import numpy as np
import pandas as pd

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [2]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

arn:aws:iam::349934754982:role/service-role/AmazonSageMaker-ExecutionRole-20190410T152971
CPU times: user 542 ms, sys: 28.8 ms, total: 571 ms
Wall time: 1.57 s


### Download News Aggregator Dataset available at the public UCI dataset repository (these files should already be downloaded in previous notebook)

In [3]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

In [4]:
#!unzip NewsAggregatorDataset.zip

In [5]:
#!rm -rf __MACOSX/

In [6]:
#ls

### Let's visualize the dataset

We will load the newsCorpora.csv file to a Pandas dataframe for our data processing work

In [7]:
import pandas as pd
import tensorflow
import re
import numpy as np
import os

In [8]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
news_dataset = pd.read_csv('newsCorpora.csv', names=column_names, header=None, delimiter='\t')
news_dataset.head()

,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


#### For this exercice we'll only use the title (Headline) of the news story and the category as our target variable

In [9]:
df=news_dataset[['TITLE',"CATEGORY"]]

In [10]:
from collections import Counter
Counter(df['CATEGORY'])

Counter({'b': 115967, 't': 108344, 'e': 152469, 'm': 45639})

The dataset has four categories: Business (b), Science & Technology (t), Entertainment (e) and Health & Medicine (m).

## Natural Language pre processing

We will do some basic processing of the text data to convert it into numerical form that the algorithm will be able to consume to create a model.
We will do typical pre processing for NLP workloads such as: dummy encoding the labels, tokenizing the documents and set fixed sequence lengths for input feature dimension, padding documents to have fixed length input vectors.

#### Dummy encode the labels

In [11]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
encoder = preprocessing.LabelEncoder()

docs = df["TITLE"].values

encoder.fit(df["CATEGORY"].values)
encoded_Y = encoder.transform(df["CATEGORY"].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

Using TensorFlow backend.


In [12]:
#bucket = <bucket> # custom bucket name.
s3_bucket = sess.default_bucket()
s3_prefix = 'news'

In [13]:
list(encoder.classes_)

['b', 'e', 'm', 't']

In [14]:
encoded_Y

array([0, 0, 0, ..., 2, 2, 2])

#### Tokenize documents and set fixed sequence lengths for input feature dimension.

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print("Vocabulary size: " + str(vocab_size))
# pad documents to a max length of 4 words
max_length = 40
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Number of headlines: " + str(len(padded_docs)))

Vocabulary size: 75286
Number of headlines: 422419


In [16]:
docs[0]

'Fed official says weak data caused by weather, should not slow taper'

### Import word embeddings

The vectors.txt file is the output of the blazingtext_word2vec_text8.ipynb notebook. This will have a list of vector representations for each word in our vocabulary.

In [17]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./vectors.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 71291 word vectors.


##### Create embedding matrix

In [18]:
#embeddings_index

In [19]:
#print(t.word_index)

In [20]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
mkdir ./data/ ./data/embeddings/

### Save embedding matrix to push to S3 for Sagemaker to use during training

In [22]:
#embedding_matrix.dump("ingredients-embedding-matrix.dat")
np.save(file="./data/embeddings/docs-embedding-matrix",
        arr=embedding_matrix,
        allow_pickle=False)
print(embedding_matrix.shape)

(75286, 100)


### Train, test split

In this section we will prep the data for ingestion for the algortihm. Split the data set in train and test samples and uplad the data to S3

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_docs, dummy_y, test_size=0.2, random_state=42)

In [24]:
!mkdir data/train/ data/test/

In [25]:
np.save('./data/train/train_X.npy', X_train)
np.save('./data/train/train_Y.npy', y_train)
np.save('./data/test/test_X.npy', X_test)
np.save('./data/test/test_Y.npy', y_test)

In [26]:
traindata_s3_prefix = '{}/data/train'.format(s3_prefix)
testdata_s3_prefix = '{}/data/test'.format(s3_prefix)
embeddings_s3_prefix='{}/data/embeddings'.format(s3_prefix)
output_s3 = 's3://{}/{}/models/'.format(s3_bucket, s3_prefix)
code_location_s3 = 's3://{}/{}/codes'.format(s3_bucket, s3_prefix)

In [27]:
train_s3 = sess.upload_data(path='./data/train/', bucket=s3_bucket, key_prefix=traindata_s3_prefix)
test_s3 = sess.upload_data(path='./data/test/', bucket=s3_bucket, key_prefix=testdata_s3_prefix)
embeddings_s3 = sess.upload_data(path='./data/embeddings/', bucket=s3_bucket, key_prefix=embeddings_s3_prefix)


In [28]:
inputs = {'train':train_s3, 'test': test_s3, 'embeddings': embeddings_s3}

print(inputs)

{'train': 's3://sagemaker-ap-southeast-1-349934754982/news/data/train', 'test': 's3://sagemaker-ap-southeast-1-349934754982/news/data/test', 'embeddings': 's3://sagemaker-ap-southeast-1-349934754982/news/data/embeddings'}


## Sagemaker training with differentiated infrastructure

We will use the high level MXNet SDK to train our MXNet model using Sagemaker. We have packaged the code we used to build and train our model in the previous notebook (headline-classifier-local.ipynb). The training script is available in the tf-src directory. You can read the details of the script and format in the python file: keras_script_mxnet.py.

In [30]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

### Define hyperparameters to push to algorithm

In [33]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':encoder.classes_.size}

We have our `TensorFlow` estimator object, we have set the hyper-parameters for this object and we have our data channels linked with the algorithm. The only  remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instance that we requested while creating the `TensorFlow` estimator classes is provisioned and is setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take some time, depending on the size of the data. 

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

We will run the training job using a ml.p2.xlarge instance (GPUs) to accelerate our training. If your account runs into resource limits please use a ml.c5.xlarge instance. 

In [ ]:
tf_estimator = TensorFlow(entry_point='keras_script.py',
                       source_dir='./tf-src',
                        role=role,
                        train_instance_count=2, 
                        train_instance_type='ml.p3.8xlarge',
                        framework_version='1.11', py_version='py3',
                        distributions={'parameter_server': {'enabled': True}},
                        hyperparameters=hyperparameters,
                         metric_definitions=[
                           {'Name': 'validation:loss', 'Regex': 'loss: ([0-9\\.]+)'},
                           {'Name': 'validation:accuracy', 'Regex': 'acc: ([0-9\\.]+)'}]
                        )
tf_estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: sagemaker-tensorflow-scriptmode-2019-04-22-11-22-51-058


2019-04-22 11:22:54 Starting - Starting the training job...
2019-04-22 11:22:56 Starting - Launching requested ML instances...
2019-04-22 11:23:34 Starting - Insufficient capacity error from EC2 while launching instances, retrying!.........
2019-04-22 11:25:07 Starting - Preparing the instances for training...
2019-04-22 11:25:53 Downloading - Downloading input data...
2019-04-22 11:26:11 Training - Downloading the training image..
2019-04-22 11:26:33,755 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-04-22 11:26:34,038 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-04-22 11:26:34,039 sagemaker_tensorflow_container.training INFO     Launching parameter server process
2019-04-22 11:26:34,039 sagemaker_tensorflow_container.training INFO     Running distributed training job with parameter servers
2019-04-22 11:26:36,055 sagemaker_tensorflow_container.training INFO     Launching wo

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 40, 100)           7528600   
_________________________________________________________________
conv_1 (Conv1D)              (None, 38, 128)           38528     
_________________________________________________________________
maxpool_1 (MaxPooling1D)     (None, 7, 128)            0         
_________________________________________________________________
flat_1 (Flatten)             (None, 896)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 896)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               114816    
_________________________________________________________________
out_1 (Dense)                (None, 4)                 516       
Total para

While the training job is running take a minute to look at the keras_script_mxnet.py script. You can see how we have adapted the our original local code from headline-classifier-mxnet.ipynb to run on Sagemaker.

## Hosting / Inference
Once the training is done, we can deploy the trained model as an Amazon SageMaker real-time hosted endpoint. This will allow us to make predictions (or inference) from the model. Note that we don't have to host on the same type of instance that we used to train. Because instance endpoints will be up and running for long, it's advisable to choose a cheaper instance for inference.

We will deploy our model using the MXNetModel object that will recieve as input the model object output by the training job to S3. We can use the different features of Sagemaker as decoupled modules. You can bring a model trained outside of Sagemaker and deploy it using the Sagemaker model deployment api.

In [ ]:
import boto3
s3 = boto3.resource('s3')

key = mxnet_estimator.model_data[mxnet_estimator.model_data.find("/", 5)+1:]
s3.Bucket(s3_bucket).download_file(key, 'model.tar.gz')

In [ ]:
model_path='model.tar.gz'
from sagemaker.mxnet import MXNet, MXNetModel

sagemaker_model = MXNetModel(model_data = model_path,
                             role = role,
                             entry_point = 'default_classifier.py',
                             py_version='py3')

In [ ]:
predictor = mxnet_estimator.deploy(initial_instance_count=1,
                           instance_type='ml.t2.medium')

### Your model should now be in production as a RESTful API!

In [ ]:
display(predictor.accept, predictor.content_type, predictor.deserializer, predictor.endpoint, predictor.sagemaker_session, predictor.serializer)

Let's evaluate our model with an example headline. You can be creative with your headlines.

In [ ]:
example_doc=['Senate prepares to vote on dueling plans to end shutdown']
# integer encode the document
encoded_example = t.texts_to_sequences(example_doc)

# pad documents to a max length of 4 words
max_length = 40
padded_example = pad_sequences(encoded_example, maxlen=max_length, padding='post')

The result represents the probability or confidence of the model that the headline represents one of the classes: 'b', 'e', 'm', 't'

In [ ]:
predictor.predict(padded_example.tolist())

In [ ]:
sess.delete_endpoint(predictor.endpoint)

## HPO - Automatic Hyperparameter tuning (Optional)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [ ]:
#hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.01, 0.2)}
hyperparameter_ranges = {'epochs': IntegerParameter(2, 7)}

In [ ]:
objective_metric_name = 'loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'loss',
                       'Regex': 'loss = ([0-9\\.]+)'}]

In [ ]:
hyperparameters = {'epochs': 5, 'vocab_size':vocab_size, 'num_classes':encoder.classes_.size}

In [ ]:
mxnet_estimator = MXNet(entry_point='keras_script_mxnet.py',
                       source_dir='./tf-src',
                        role=role,
                        train_instance_type='ml.c5.xlarge',
                        train_instance_count=1,
                        framework_version='1.3.0',
                        py_version='py3',
                        hyperparameters=hyperparameters)

In [ ]:
tuner = HyperparameterTuner(mxnet_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=5,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

In [ ]:
tuner.fit(inputs)

Go to Sagemaker console and under Training > Hyperparameter Tuning jobs > select the only entry. You will see different training jobs  with model configurations as well as a "Best Training job" tab.